In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# cd drive/MyDrive/따릉이

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings('ignore')


plt.rc('font', family='Malgun Gothic')

In [ ]:
station_df = pd.read_csv('data/station_data.csv', index_col=0)
break_df = pd.read_csv('data/서울시 공공자전거 고장신고 내역_23.1-6.csv', encoding="cp949")
office_df = pd.read_csv('data/서울특별시 공공자전거 대여소별 이용정보(월별)_23.1-6.csv', encoding="cp949")
history_df = pd.read_csv('data/서울특별시 공공자전거 대여이력 정보_2301.csv', encoding="cp949")
history_df['반납대여소번호'] = history_df['반납대여소번호'].replace("\\N", np.nan)
history_df['반납대여소번호'] = history_df['반납대여소번호'].astype(float)
office_df['대여건수'] = office_df['대여건수'].str.replace(",", "").astype(int)
office_df['반납건수'] = office_df['반납건수'].str.replace(",", "").replace(' - ', 0).astype(int)

### 타겟 광고를 위한 데이터 분석
- 20대 남성

In [ ]:
import datetime

In [ ]:
history_df['성별'] = history_df['성별'].replace({'m': 'M', 'f': 'F', np.nan:'\\N'})

In [ ]:
history_df['생년'] = history_df['생년'].replace('\\N', np.nan)
history_df['생년'] = history_df['생년'].astype(float)

In [ ]:
current_year = datetime.datetime.now().year

filtered_history_df = history_df.copy()
filtered_history_df = filtered_history_df[(filtered_history_df['생년'] > 0) & (filtered_history_df['생년'] <= 2010)]

filtered_history_df['연령'] = current_year - filtered_history_df['생년']

age_bins = [13, 20, 30, 40, 50, 60, 70, 80, 90, 100]
age_labels = ['13-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90+']

filtered_history_df['연령대'] = pd.cut(filtered_history_df['연령'], bins=age_bins, labels=age_labels, right=False)

In [ ]:
# 20대 남성이 자주 이용하는 대여소
target_df = filtered_history_df[
    (filtered_history_df['성별'] == 'M') &
    (filtered_history_df['연령대'] == '20-29')
]
target_df.head()

In [ ]:
target_station_counts = target_df.groupby('대여 대여소명').size().reset_index(name='대여횟수')
target_station_counts.sort_values('대여횟수', ascending=False).head(10)

In [ ]:
total_rentals_per_station = filtered_history_df.groupby('대여 대여소명').size().reset_index(name='전체대여횟수')
total_rentals_per_station.head()

In [ ]:
merged_df = total_rentals_per_station.merge(target_station_counts, on='대여 대여소명', how='left')
merged_df.head()

In [ ]:
merged_df['20대남성비율'] = merged_df['대여횟수']/merged_df['전체대여횟수']
merged_df.sort_values('20대남성비율', ascending=False).head(10)

In [ ]:
total_rentals = len(filtered_history_df)
target_rentals = len(target_df)

target_rentals / total_rentals

### KMeans를 활용한 비슷한 성향의 대여소 찾기
- 서울과학기술대학교(어학교육원)과 비슷한 성향의 대여소 찾기

In [ ]:
filtered_history_df['대여시간'] = pd.to_datetime(filtered_history_df['대여일시']).dt.hour

In [ ]:
hourly_groupby_df = filtered_history_df.groupby(['대여 대여소명', '대여시간']).size().unstack(fill_value=0)
hourly_groupby_df.head()

In [ ]:
age_groupby_df = filtered_history_df.groupby(['대여 대여소명', '연령대']).size().unstack(fill_value=0)
age_groupby_df.head()

In [ ]:
gender_groupby_df = filtered_history_df.groupby(['대여 대여소명', '성별']).size().unstack(fill_value=0)
gender_groupby_df.head()

In [ ]:
general_groupby_df = filtered_history_df.groupby(['대여 대여소명', '성별', '연령대', '대여시간']).size()
general_groupby_df = general_groupby_df.unstack("성별")
general_groupby_df = general_groupby_df.unstack("연령대")
general_groupby_df = general_groupby_df.unstack("대여시간")
general_groupby_df.head()

In [ ]:
general_groupby_df.columns = [" ".join(
    [str(c) for c in column]
) for column in general_groupby_df.columns]

In [ ]:
general_groupby_df

In [ ]:
features_df = pd.concat([general_groupby_df, gender_groupby_df, age_groupby_df, hourly_groupby_df], axis=1)
features_df

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
features_df.columns = features_df.columns.astype(str)

In [ ]:
kmeans = KMeans(n_clusters=30, random_state=13)
features_df['cluster'] = kmeans.fit_predict(features_df)
features_df

In [ ]:
cluster = features_df.loc['서울과학기술대학교(어학교육원)']['cluster']
features_df[features_df['cluster'] == cluster].index

In [ ]:
len(features_df[features_df['cluster'] == cluster].index)